## Problem Statement/Goal



## Imports

In [65]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np
%matplotlib inline
from selenium import webdriver
import requests
from time import sleep
from bs4 import BeautifulSoup
import json

from sklearn.model_selection import train_test_split

# Selenium

# Translating Selenium into Requests & Soup

In [66]:
def get_html(url):
    return BeautifulSoup(requests.get(url).content, 'lxml')

In [67]:
# Function to specifically spit out a list of tuples
# tuple[0] is the url of the topic
# tuple[1] is the number of pages in the topic
def urls_with_numbers(forum_page_url):
#current_forum_page = 'https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page)
    forum_soup = get_html(forum_page_url)                                       # Input URL get Soup
    topic_url_list = ['https://us.battle.net' + topic.attrs['href'] for topic in forum_soup.find_all(attrs={'class': "ForumTopic"})]
    topic_tuples = []                                                           # Above is list of urls of topics in forum page
    count = 0                                                                   # Instantiating empty list and starting count
    for forum_topic in forum_soup.find_all(attrs={'class': "ForumTopic"}):      # for i in all th3e forum topic infos
        posts_num = json.loads(forum_topic.attrs['data-forum-topic'])['lastPosition'] # Turning each individual info bit into a dict
        topic_pages = posts_num//20 + 1                     # Number of posts in the Topic divided by num allowed per page +1 for 1st
        if posts_num%20 != 0 and posts_num > 20:            # If there's a remainder page
            topic_pages += 1                                # Add remainder page
        topic_tuples.append((topic_url_list[count],topic_pages)) # Add the url to the page amount in tuples
        count += 1                                          # Keep track of the count
    return topic_tuples                                     # Returns list of tuples


In [68]:
def save_posts(list_of_dicts, saver, forum_page):
    return pd.DataFrame(list_of_dicts, columns = ['text',
                            'date',
                            'ids_dict',
                            'post_num',
                            'auth_posts',
                            'prof_link',
                            'title',
                            'forum_page',
                            'stqatuses',
                            'topic_url']).to_csv(saver + str(forum_page), index=False)

In [ ]:
words = []
dates = []
TopicPosts = []
title = []
forum_page = []
prof_link = []
auth_posts = []
topic_keys  = []
def turn_to_dict(post, words = words, 
                 dates = dates, TopicPosts = TopicPosts, 
                 title = title, forum_page = forum_page, 
                 prof_link = prof_link, auth_posts = auth_posts, topic_keys = topic_keys):
    post_dict = {"If you're seeing this": "something went wrong."}
    try:
        post_dict = {                               # Creation & Statement of dicts
            'text'      : words[post].contents,                     # Text of the post
            'date'      : dates[post].attrs['data-tooltip-content'],# Date of the (unedited) post
            'ids_dict'  : TopicPosts[post].attrs['data-topic-post'],# Author info & votes of the post
            'post_num'  : TopicPosts[post].attrs['id'] ,            # Post number in the topic
            'title'     : title,                    # Title of Topic
            'forum_page': forum_page,               # Page in the forum
            'topic_url' : topic_keys[post][0]}
    except:
        pass
    
    try:
        post_dict['statuses']   = TopicPosts[post].attrs['data-topic']
    except:
        pass
    
    try:
        post_dict['prof_link']  = prof_link[post].attrs['href'][0]
    except:
        pass
    
    try:
        post_dict['auth_posts'] = auth_posts[post][0]  # Number of posts author has made
    except:
        pass
    
    return post_dict

In [70]:
edt = ['\n\t\t\t\t\t\t\t\xa0(Edited)\n']

In [71]:
# Check that both forum_page 0 & 1 work without causing doubling up or other issues

In [80]:
# THIS is for turning into the Requests library
list_of_dicts = []                                              # Instantiate empty list of DICKS to FILL UP
saver = './data/full_scrapes/Overwatch_Test_'                   # The path & starting name for saving junk

for forum_page in range(101):                                 # Will go through all the forum pages specified
    topic_keys = urls_with_numbers('https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page))
    if forum_page % 100 == 0:
        save_posts(list_of_dicts, saver, forum_page) 
    for topic_stats in topic_keys:                              # For every title in the list
        for page in range(topic_stats[1]):                      # for page # in range of # of pages in the topic
            #page += 1                                           # Add 1 to compensate for starting at 0
            if page > 1:                                        # Basically making sure it's not the first page
                URL = topic_stats[0] + '?page=' + str(page)     # This is basicslly for getting to all the pages
            else:
                URL = topic_stats[0]                            # This will be the first page
            sleep(5)
            
            # This chunk defines the lists of things that we want per row
            page_soup  = get_html(URL)                          # Soup of the page
            title      = page_soup.find(attrs={'class':'Topic-title'}).contents
            dates      = [e for e in page_soup.find_all('a',{"class" :'TopicPost-timestamp'}) if e.contents !=edt]
            words      = page_soup.find_all('div',{"class" :'TopicPost-bodyContent'})
            TopicPosts = page_soup.find_all(attrs = {"class" :'TopicPost'})
            auth_posts = [e.contents for e in page_soup.find_all('a',attrs = {'class' :'Author-posts'})]
            prof_link  = page_soup.find_all(attrs = {"class" :'Author-avatar '})
            print("F-Page:", forum_page, "Posts/Topic:", topic_stats[1],"Page:",page,"URL:",URL )
            if page == 1:
                print("Skipped duplicate page")
            else:
                for post in range(len(words)):                           # For each post that's present
                    list_of_dicts.append(turn_to_dict(post, words, 
                 dates, TopicPosts, title, forum_page, 
                 prof_link, auth_posts, topic_keys))        # Add the dict of post to list of posts
        print("END F-page:", forum_page,"Topic Page:", page, "Last URL:", URL) # Sanity check
df = save_posts(list_of_dicts, saver, forum_page)               # Should both save the data & create a df to see

F-Page: 0 Posts/Topic: 1 Page: 0 URL: https://us.battle.net/forums/en/overwatch/topic/20744265634
END F-page: 0 Topic Page: 0 Last URL: https://us.battle.net/forums/en/overwatch/topic/20744265634
F-Page: 0 Posts/Topic: 1 Page: 0 URL: https://us.battle.net/forums/en/overwatch/topic/20744215778
END F-page: 0 Topic Page: 0 Last URL: https://us.battle.net/forums/en/overwatch/topic/20744215778
F-Page: 0 Posts/Topic: 238 Page: 0 URL: https://us.battle.net/forums/en/overwatch/topic/20761088072
F-Page: 0 Posts/Topic: 238 Page: 1 URL: https://us.battle.net/forums/en/overwatch/topic/20761088072
Skipped duplicate page
F-Page: 0 Posts/Topic: 238 Page: 2 URL: https://us.battle.net/forums/en/overwatch/topic/20761088072?page=2
F-Page: 0 Posts/Topic: 238 Page: 3 URL: https://us.battle.net/forums/en/overwatch/topic/20761088072?page=3
F-Page: 0 Posts/Topic: 238 Page: 4 URL: https://us.battle.net/forums/en/overwatch/topic/20761088072?page=4
F-Page: 0 Posts/Topic: 238 Page: 5 URL: https://us.battle.net/fo

KeyboardInterrupt: 

In [82]:
df = save_posts(list_of_dicts, saver, forum_page)

In [98]:
TopicPosts[1].attrs#['data-topic']

{'class': ['TopicPost', ''],
 'id': 'post-142',
 'data-topic-post': '{"id":"207593303592","valueVoted":0,"rank":{"voteUp":20,"voteDown":4},"author":{"id":"207480975916","name":"TaliMagpie"}}'}

In [87]:
pd.DataFrame(list_of_dicts, columns = ['text',
                            'date',
                            'ids_dict',
                            'post_num',
                            'auth_posts',
                            'prof_link',
                            'title',
                            'forum_page',
                            'statuses',
                            'topic_url'])

,text,date,ids_dict,post_num,auth_posts,prof_link,title,forum_page,statuses,topic_url
0,"[Welcome to the , <strong>General Discussion</...",05/23/2016 09:32 PM,"{""id"":""207423780179"",""valueVoted"":0,""rank"":{""v...",post-1,NaN,NaN,[Welcome to General Discussion - Please Read!],0,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...
1,"[In addition to the forum , <a class=""bml-link...",05/23/2016 09:33 PM,"{""id"":""207423980450"",""valueVoted"":0,""rank"":{""v...",post-2,NaN,NaN,[Welcome to General Discussion - Please Read!],0,NaN,https://us.battle.net/forums/en/overwatch/topi...
2,[To help us get the most of out of your feedba...,05/23/2016 09:36 PM,"{""id"":""207423481262"",""valueVoted"":0,""rank"":{""v...",post-1,NaN,NaN,[Bug Report and Technical Support Forums],0,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...
3,"[Hi everyone, , <br/>, <br/>, We've continued ...",01/30/2018 09:21 PM,"{""id"":""207594900856"",""valueVoted"":0,""rank"":{""v...",post-1,[ 368 posts ],https://playoverwatch.com/en-us/career/pc/us/M...,"[[Feedback Thread] Mercy Updates - Jan 30, 2018]",0,"{ ""sticky"":""true"",""featured"":""false"",""locked"":...",https://us.battle.net/forums/en/overwatch/topi...
4,[Please try to see the Valkyrie rework didn't ...,01/30/2018 09:25 PM,"{""id"":""207595100830"",""valueVoted"":0,""rank"":{""v...",post-2,[ 17 posts ],https://playoverwatch.com/en-us/career/pc/us/O...,"[[Feedback Thread] Mercy Updates - Jan 30, 2018]",0,NaN,https://us.battle.net/forums/en/overwatch/topi...
5,[Ban-hammer much? Surprised you guys didnt re...,01/30/2018 09:26 PM,"{""id"":""207592902925"",""valueVoted"":0,""rank"":{""v...",post-3,[ 7 posts ],https://playoverwatch.com/en-us/career/pc/us/M...,"[[Feedback Thread] Mercy Updates - Jan 30, 2018]",0,NaN,https://us.battle.net/forums/en/overwatch/topi...
6,[You guys are joking with this right? The othe...,01/30/2018 09:27 PM,"{""id"":""207593602686"",""valueVoted"":0,""rank"":{""v...",post-4,[ 17 posts ],https://playoverwatch.com/en-us/career/pc/us/A...,"[[Feedback Thread] Mercy Updates - Jan 30, 2018]",0,NaN,https://us.battle.net/forums/en/overwatch/topi...
7,[tldr: Mercy has no Ult anymore],01/30/2018 09:28 PM,"{""id"":""207593402413"",""valueVoted"":0,""rank"":{""v...",post-5,[ 353 posts ],https://playoverwatch.com/en-us/career/pc/us/E...,"[[Feedback Thread] Mercy Updates - Jan 30, 2018]",0,NaN,https://us.battle.net/forums/en/overwatch/topi...
8,[You took in tons of feedback and still ignore...,01/30/2018 09:29 PM,"{""id"":""207594302097"",""valueVoted"":0,""rank"":{""v...",post-6,[ 504 posts ],https://playoverwatch.com/en-us/career/pc/us/W...,"[[Feedback Thread] Mercy Updates - Jan 30, 2018]",0,NaN,https://us.battle.net/forums/en/overwatch/topi...
9,"[<blockquote class=""quote-public"" data-quote=""...",01/30/2018 09:29 PM,"{""id"":""207593703439"",""valueVoted"":0,""rank"":{""v...",post-7,[ 6 posts ],https://playoverwatch.com/en-us/career/pc/us/E...,"[[Feedback Thread] Mercy Updates - Jan 30, 2018]",0,NaN,https://us.battle.net/forums/en/overwatch/topi...


In [81]:
list_of_dicts

[{'text': ['Welcome to the ',
   <strong>General Discussion</strong>,
   ' forum! We encourage you to use this forum to provide feedback and/or discuss your experiences while playing the Overwatch.',
   <br/>,
   <br/>,
   'Community forums work best when participants treat their fellow posters with respect and courtesy, so we ask that you take the time to read through the forum Code of Conduct and Forum Guidelines before posting.',
   <br/>,
   <br/>,
   <ul><li><strong><a class="bml-link-url2" href="http://us.battle.net/en/community/conduct">Code of Conduct</a></strong> </li><li><strong><a class="bml-link-url2" href="http://us.battle.net/forums/en/overwatch/topic/20744265634?page=1#post-2">Forum Guidelines</a></strong> </li></ul>,
   <br/>,
   'For the latest Overwatch news, ',
   <a class="bml-link-url2" href="https://playoverwatch.com/en-us/blog/">click here</a>,
   '.',
   <br/>,
   'For the latest Overwatch patch notes, ',
   <a class="bml-link-url2" href="http://us.battle.net/fo

In [453]:
res = requests.get(current_list)

In [457]:
res.content

b'\n\n\n<html xmlns="http://www.w3.org/1999/xhtml">\n\n<head>\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge"/>\n\t<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no"/>\n\n\t\n\n\t<title>General Discussion - Overwatch Forums</title>\n\n\t<meta property="og:type" content="website" />\n\t<meta property="og:url" content="https://us.battle.net/forums/en/overwatch/22813879/?page=72" />\n\t<meta property="og:title" content="General Discussion - Overwatch Forums" />\n\t\t<meta property="og:image" content="/forums/static/images/social-thumbs/overwatch.png" />\n\n\t\t<meta property="og:description" content="Discuss all things Overwatch with fellow players." />\n\t\thttp://us.battle.net/overwatch/en/forum/22813879/\n\n\n\t\t<link rel="shortcut icon" type="image/x-icon" href="/forums/static/images/icons/overwatch-favicon.ico" />\n\t\t\t<link rel="stylesheet" type="text/css" media="all" href="/forums/static/css/navbar.css?v=58-88" />\r\n\n

In [456]:
res.text

'\n\n\n<html xmlns="http://www.w3.org/1999/xhtml">\n\n<head>\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge"/>\n\t<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no"/>\n\n\t\n\n\t<title>General Discussion - Overwatch Forums</title>\n\n\t<meta property="og:type" content="website" />\n\t<meta property="og:url" content="https://us.battle.net/forums/en/overwatch/22813879/?page=72" />\n\t<meta property="og:title" content="General Discussion - Overwatch Forums" />\n\t\t<meta property="og:image" content="/forums/static/images/social-thumbs/overwatch.png" />\n\n\t\t<meta property="og:description" content="Discuss all things Overwatch with fellow players." />\n\t\thttp://us.battle.net/overwatch/en/forum/22813879/\n\n\n\t\t<link rel="shortcut icon" type="image/x-icon" href="/forums/static/images/icons/overwatch-favicon.ico" />\n\t\t\t<link rel="stylesheet" type="text/css" media="all" href="/forums/static/css/navbar.css?v=58-88" />\r\n\n\

In [459]:
resoup = BeautifulSoup(res.content, 'lxml')

TypeError: replace() takes at least 2 arguments (0 given)

In [503]:
resoup

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport"/>
<title>General Discussion - Overwatch Forums</title>
<meta content="website" property="og:type"/>
<meta content="https://us.battle.net/forums/en/overwatch/22813879/?page=72" property="og:url"/>
<meta content="General Discussion - Overwatch Forums" property="og:title"/>
<meta content="/forums/static/images/social-thumbs/overwatch.png" property="og:image"/>
<meta content="Discuss all things Overwatch with fellow players." property="og:description"/></head><body><p>
		http://us.battle.net/overwatch/en/forum/22813879/


		<link href="/forums/static/images/icons/overwatch-favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/forums/static/css/navbar.css?v=58-88" media="all" rel="stylesheet" type="text/css"/>
<link href="/forums/static/css/main-dd90a936ce.css?v=58-88" me

In [7]:
test[0].attrs

{'class': ['ForumTopic',
  'ForumTopic--sticky',
  'has-blizzard-post',
  'is-locked'],
 'href': '/forums/en/overwatch/topic/20744265634',
 'data-forum-topic': '{"id":20744265634,"lastPosition":1,"isSticky":true,"isFeatured":false,"isLocked":true,"isHidden":false,"isSpam":false}',
 'data-created-date': '5/23/16',
 'data-created-time': '21:32',
 'data-creator': ''}

In [ ]:
print(test[i].attrs['data-forum-topic']

In [511]:
list_of_topic_links = ['https://us.battle.net' + topic.attrs['href'] for topic in resoup.find_all(attrs={'class': "ForumTopic"})]

In [11]:
4736//20


236

In [23]:
test = resoup.find_all(attrs={'class': "ForumTopic"})
for i in range(len(test)):
    print('-------')
    print(i)
    posts = json.loads(test[i].attrs['data-forum-topic'])['lastPosition']
    print('posts:', posts)
    pages = posts//20 + 1
    print("pages:", pages)
    if posts%20 > 0 and posts > 21:
        pages += 1
        print("has remainder", posts%20, 'pages:', pages)
    

-------
0
posts: 1
pages: 1
-------
1
posts: 0
pages: 1
-------
2
posts: 4736
pages: 237
has remainder 16 pages: 238
-------
3
posts: 89
pages: 5
has remainder 9 pages: 6
-------
4
posts: 161
pages: 9
has remainder 1 pages: 10
-------
5
posts: 17
pages: 1
-------
6
posts: 7
pages: 1
-------
7
posts: 18
pages: 1
-------
8
posts: 14
pages: 1
-------
9
posts: 30
pages: 2
has remainder 10 pages: 3
-------
10
posts: 25
pages: 2
has remainder 5 pages: 3
-------
11
posts: 5
pages: 1
-------
12
posts: 17
pages: 1
-------
13
posts: 3
pages: 1
-------
14
posts: 9
pages: 1
-------
15
posts: 8
pages: 1
-------
16
posts: 22
pages: 2
has remainder 2 pages: 3
-------
17
posts: 12
pages: 1
-------
18
posts: 6
pages: 1
-------
19
posts: 0
pages: 1
-------
20
posts: 6244
pages: 313
has remainder 4 pages: 314
-------
21
posts: 37
pages: 2
has remainder 17 pages: 3
-------
22
posts: 14
pages: 1
-------
23
posts: 6
pages: 1
-------
24
posts: 8
pages: 1
-------
25
posts: 6
pages: 1
-------
26
posts: 0
pages

In [23]:
test = resoup.find_all(attrs={'class': "ForumTopic"})
for i in range(len(test)):
    posts = json.loads(test[i].attrs['data-forum-topic'])['lastPosition']
    pages = posts//20 + 1
    if posts%20 > 0 and posts > 21:
        pages += 1
for  in test

-------
0
posts: 1
pages: 1
-------
1
posts: 0
pages: 1
-------
2
posts: 4736
pages: 237
has remainder 16 pages: 238
-------
3
posts: 89
pages: 5
has remainder 9 pages: 6
-------
4
posts: 161
pages: 9
has remainder 1 pages: 10
-------
5
posts: 17
pages: 1
-------
6
posts: 7
pages: 1
-------
7
posts: 18
pages: 1
-------
8
posts: 14
pages: 1
-------
9
posts: 30
pages: 2
has remainder 10 pages: 3
-------
10
posts: 25
pages: 2
has remainder 5 pages: 3
-------
11
posts: 5
pages: 1
-------
12
posts: 17
pages: 1
-------
13
posts: 3
pages: 1
-------
14
posts: 9
pages: 1
-------
15
posts: 8
pages: 1
-------
16
posts: 22
pages: 2
has remainder 2 pages: 3
-------
17
posts: 12
pages: 1
-------
18
posts: 6
pages: 1
-------
19
posts: 0
pages: 1
-------
20
posts: 6244
pages: 313
has remainder 4 pages: 314
-------
21
posts: 37
pages: 2
has remainder 17 pages: 3
-------
22
posts: 14
pages: 1
-------
23
posts: 6
pages: 1
-------
24
posts: 8
pages: 1
-------
25
posts: 6
pages: 1
-------
26
posts: 0
pages

In [29]:
topic_tuples[2][1]

238

In [26]:
topic_url_list

['https://us.battle.net/forums/en/overwatch/topic/20744265634',
 'https://us.battle.net/forums/en/overwatch/topic/20744215778',
 'https://us.battle.net/forums/en/overwatch/topic/20761088072',
 'https://us.battle.net/forums/en/overwatch/topic/20761806957',
 'https://us.battle.net/forums/en/overwatch/topic/20761046611',
 'https://us.battle.net/forums/en/overwatch/topic/20761916683',
 'https://us.battle.net/forums/en/overwatch/topic/20762066698',
 'https://us.battle.net/forums/en/overwatch/topic/20762026641',
 'https://us.battle.net/forums/en/overwatch/topic/20761916686',
 'https://us.battle.net/forums/en/overwatch/topic/20762056694',
 'https://us.battle.net/forums/en/overwatch/topic/20749827037',
 'https://us.battle.net/forums/en/overwatch/topic/20761886659',
 'https://us.battle.net/forums/en/overwatch/topic/20761966681',
 'https://us.battle.net/forums/en/overwatch/topic/20761976664',
 'https://us.battle.net/forums/en/overwatch/topic/20761846667',
 'https://us.battle.net/forums/en/overwa

In [ ]:
# obtain topic number in touple pair with the url
# take the posts number if it's >0 add all posts to list
# then go to next page

In [474]:
resoup
"""<a class="ForumTopic" data-created-date="2/2/18" data-created-time="15:43" data-creator="Direwolf20#1179" data-forum-topic='{"id":20761776225,"lastPosition":48,"isSticky":false,"isFeatured":false,"isLocked":false,"isHidden":false,"isSpam":false}' href="/forums/en/overwatch/topic/20761776225"> <span class="ForumTopic-type">
<i class="Icon"></i>
</span><div class="ForumTopic-details"><span class="ForumTopic-heading"> <span class="ForumTopic-title--wrapper">
<span class="ForumTopic-timestamp on-mobile">
<a class="ForumTopic-timestamp--lastPost" data-toggle="tooltip" data-tooltip-content="Last Post" href="/forums/en/overwatch/topic/20761776225?page=3#post-49">
Feb 16
					</a>"""

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport"/>
<title>General Discussion - Overwatch Forums</title>
<meta content="website" property="og:type"/>
<meta content="https://us.battle.net/forums/en/overwatch/22813879/?page=72" property="og:url"/>
<meta content="General Discussion - Overwatch Forums" property="og:title"/>
<meta content="/forums/static/images/social-thumbs/overwatch.png" property="og:image"/>
<meta content="Discuss all things Overwatch with fellow players." property="og:description"/></head><body><p>
		http://us.battle.net/overwatch/en/forum/22813879/


		<link href="/forums/static/images/icons/overwatch-favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/forums/static/css/navbar.css?v=58-88" media="all" rel="stylesheet" type="text/css"/>
<link href="/forums/static/css/main-dd90a936ce.css?v=58-88" me

In [444]:
soup.find(attrs = {'class':'error-type'}).contents[0]

'404'

In [449]:
save_posts(list_of_dicts, saver, forum_page)

In [267]:
#current_list = browser.get('https://us.battle.net/forums/en/overwatch/22813879/?page=5')

In [319]:
soup = BeautifulSoup(browser.page_source, 'lxml')

In [198]:
pd.DataFrame(list_of_dicts, 
                         columns = ['text','date'])

,text,date
0,Thank you to the tanks who protect your team.T...,\n\t\t\t\t\t\tFeb 20\n\t\t\t\t\t
1,Fck d.va and mercy mains.IM OUT!,\n\t\t\t\t\t\tFeb 20\n\t\t\t\t\t
2,02/20/2018 11:30 AMPosted by HayaishiFck d.va ...,\n\t\t\t\t\t\tFeb 20\n\t\t\t\t\t
3,02/20/2018 11:30 AMPosted by HayaishiFck d.va ...,\n\t\t\t\t\t\tFeb 20\n\t\t\t\t\t
4,02/20/2018 11:30 AMPosted by HayaishiFck d.va ...,\n\t\t\t\t\t\tFeb 20\n\t\t\t\t\t
5,02/20/2018 11:30 AMPosted by HayaishiFck d.va ...,\n\t\t\t\t\t\tFeb 20\n\t\t\t\t\t
6,02/20/2018 11:29 AMPosted by RythixThank you t...,\n\t\t\t\t\t\tFeb 20\n\t\t\t\t\t
7,"To, sorry Jelly is late to the party. Got hel...",\n\t\t\t\t\t\tFeb 20\n\t\t\t\t\t
8,"02/20/2018 11:35 AMPosted by JellyandJamTo, so...",\n\t\t\t\t\t\tFeb 20\n\t\t\t\t\t
9,Why do I have a feeling that the final post on...,\n\t\t\t\t\t\tFeb 20\n\t\t\t\t\t


In [184]:
soup = BeautifulSoup(browser.page_source, 'lxml')         # Soup of words
words = soup.find_all('div',{"class" :'TopicPost-bodyContent'})
dates = soup.find_all('a',{"class" :'TopicPost-timestamp'})

print(len(words))
print(len(dates))

20
22


In [409]:
# Testing Code # 1
path_to_chromedriver = '../Garage/chromedriver'                          # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path_to_chromedriver)       # Open browser
current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page=9'
browser.get(current_list)
title_list = browser.find_elements_by_class_name("ForumTopic-title")
title_list[4].click()
browser.find_element_by_class_name('Topic-title').text

TimeoutException: Message: timeout
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [409]:
# Testing Code # 1
path_to_chromedriver = '../Garage/chromedriver'                          # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path_to_chromedriver)       # Open browser
current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page=9'
browser.get(current_list)
title_list = browser.find_elements_by_class_name("ForumTopic-title")
title_list[4].click()
browser.find_element_by_class_name('Topic-title').text

TimeoutException: Message: timeout
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [426]:
# Testing Code # 2
path_to_chromedriver = '../Garage/chromedriver'                          # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path_to_chromedriver)       # Open browser
browser.get(current_topic + '?page=' + str(next_page))
soup = BeautifulSoup(browser.page_source, 'lxml')
#soup.find_all('div',{"class" :'TopicPost-bodyContent'})

In [429]:
soup.find(attrs = {'class':'error-type'}).contents[0]

'404'

In [411]:
soup = BeautifulSoup(browser.page_source, 'lxml')

TimeoutException: Message: timeout
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [140]:
# Moves down the forum, clicking on each individual title link
title_list = browser.find_elements_by_class_name("ForumTopic-title")
for i in range(len(title_list)):
    print(title_list[i])
    title_list[i].click()
    sleep(1)
    browser.back()
    title_list = browser.find_elements_by_class_name("ForumTopic-title")
    print(title_list[i])

<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.9641869846177666-1")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.3229329085227737-1")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.3229329085227737-2")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.3916746129055908-2")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.3916746129055908-3")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.9025396260201077-3")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.9025396260201077-4")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.787543

KeyboardInterrupt: 

In [409]:
# Testing Code # 1
path_to_chromedriver = '../Garage/chromedriver'                          # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path_to_chromedriver)       # Open browser
current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page=9'
browser.get(current_list)
title_list = browser.find_elements_by_class_name("ForumTopic-title")
title_list[4].click()
browser.find_element_by_class_name('Topic-title').text

TimeoutException: Message: timeout
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [182]:
soup.find_all('div',{"class" :'TopicPost-bodyContent'})[4].text

"05/23/2016 05:44 PMPosted by DakrathAre you kidding me, Blizzard? After all the changes you made between the second and third BETA sequences, I actually thought you guys cared about balancing your game. Bastion is still WAY TOO STRONK. His turret form has way too much tankiness and range for how much absolutely insane damage he does. Yes, good job taking away his shield. He did NOT, NOT, NOT need to be able to swivel 360 unrestricted (doesn't even have to backtrack once he hits a certain point) and the speed at which he can turn means that he might as well be able to shoot in every direction. If the balance on this game isn't fixed within the first 2 weeks of release, I'm asking for a refund. Not paying for another broken game. At least not quietly.if you're still complaining about this you obviously haven't learned what countering is yet"

In [177]:
soup.find_all('a',{"class" :'TopicPost-timestamp'})[2].text

'\n\t\t\t\t\t\t\t\xa0(Edited)\n'

In [171]:
# Numbers of ups or downs
soup.find_all('span',{"class" :'TopicPost-rank'})[2].text

''

In [170]:
browser.find_elements_by_class_name("TopicPost-timestamp")

[<selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="0.601834304827114-1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="0.601834304827114-2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="0.601834304827114-3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="0.601834304827114-4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="0.601834304827114-5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="0.601834304827114-6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="0.601834304827114-7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="022625332a26ac2d6e8625cba583c36b", element="

http://cryptocats.thetwentysix.io/
    1.) Scrape entire page of cats including
        Cat #
        Cat Status (claimed or not)
        Cat URL
    2.)Identify the navigation items "origin 12", Austrailian Blockchain" and "December"
    3.) Select each of these buttons, automate a click action to each of them
    4.) Run your scraping process from #1, appending the "release type", corresponding to each of these release filters

In [21]:
#dfp01 = pd.read_csv('./data/Finalish Scrapes/Overwatch_24to77_posts_75.csv')
dfp02 = pd.read_csv('./data/Finalish Scrapes/Overwatch_24to77_posts_77.csv')
#dfp03 = pd.read_csv('./data/Finalish Scrapes/Overwatch_posts_100')
#dfp04 = pd.read_csv('./data/Finalish Scrapes/Overwatch_posts_125')
dfp05 = pd.read_csv('./data/Finalish Scrapes/Overwatch_posts_149')
#dfp06 = pd.read_csv('./data/Finalish Scrapes/Overwatch_SAMPLE150to1000_950')
dfp07 = pd.read_csv('./data/Finalish Scrapes/Overwatch_SAMPLE150to1000_990')
dfp08 = pd.read_csv('./data/Finalish Scrapes/Overwatch_SAMPLEposts_1500')
#dfp09 = pd.read_csv('./data/Finalish Scrapes/Overwatch_SAMPLEposts_1520')
#dfp10 = pd.read_csv('./data/Finalish Scrapes/Overwatch_SAMPLEposts_1530')
dfp11 = pd.read_csv('./data/Finalish Scrapes/Overwatch_Sample1500to5k_5000')
#dfp12 = pd.read_csv('./data/Finalish Scrapes/Overwatch_Sample5kto10k_7200')
dfp13 = pd.read_csv('./data/Finalish Scrapes/Overwatch_Sample5kto10k_7250')

In [53]:
#df_24to77     = pd.read_csv('./data/Overwatch_24to77_posts_77.csv')
#df_149        = pd.read_csv('./data/Overwatch_posts_149')          # An error skipped pages 24 to 77
df_s0to150    = pd.read_csv('./data/Overwatch_Sample_to1500_160')
df_s150to1000 = pd.read_csv('./data/Overwatch_SAMPLE150to1000_990')
df_s1500      = pd.read_csv('./data/Overwatch_SAMPLEposts_1500')
df_s1500to5k  = pd.read_csv('./data/Overwatch_Sample1500to5k_5000')
df_s5kto10k   = pd.read_csv('./data/Overwatch_Sample5kto10k_7250')

In [56]:
df_s0to150['date']      = df_s0to150['date'].map(lambda x: x.replace('\t','').replace('\n',''))
df_s150to1000['date']   = df_s150to1000['date'].map(lambda x: x.replace('\t','').replace('\n',''))
df_s1500['date']        = df_s1500['date'].map(lambda x: x.replace('\t','').replace('\n',''))
df_s1500to5k['date']    = df_s1500to5k['date'].map(lambda x: x.replace('\t','').replace('\n',''))
df_s5kto10k['date']     = df_s5kto10k['date'].map(lambda x: x.replace('\t','').replace('\n',''))

In [57]:
dfs_listed = [    
    df_s0to150,
    df_s150to1000,
    df_s1500,
    df_s1500to5k,
    df_s5kto10k]

In [86]:

for text in df_s150to1000['text']:
    try:
        if '<Removed by forum moderator for toxicity>' in text:
            print(text)
    except:
        print(text)
        #print(df_s0to150['date'])

<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
nan
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>


In [92]:
for df in dfs_listed:
    for text in df['text']:
        try:
            if '<Removed' in text:
                df.head(1)
                print(text)
        except:
            pass

<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed>------------Removed by moderator for toxicity. Please adhere to the code of conduct when posting. Toxicity in all forms will result in suspension of posting privileges.
01/19/2018 08:57 AMPosted by Makenai<span class="truncated">...</span>Stop crying omfg, we're not considered as "fake acoount" we just didn't had any interest to login to the forums. we just base our opinions about this lame  no skill required hero.Does your mom know you use that kind of language online?Well she does you know<Removed>----------------------Edited by moderator for language. Please adhere to the code of conduct when posting.
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<Removed by forum moderator for toxicity>
<

In [64]:
df_s150to1000.tail(20)

,text,date
32864,Its about time,Dec 19
32865,12/19/2017 01:19 PMPosted by n0b0dyIts about t...,Dec 19
32866,12/19/2017 01:21 PMPosted by iamhappyso12/19/2...,Dec 19
32867,12/19/2017 01:23 PMPosted by Mamalamadoo12/19/...,Dec 19
32868,He needs a nerf to the amount of forum nerf re...,Dec 19
32869,12/19/2017 01:25 PMPosted by DoctorArmondHe ne...,Dec 19
32870,12/19/2017 01:19 PMPosted by n0b0dyIts about t...,Dec 19
32871,12/19/2017 01:30 PMPosted by DrgHybrid12/19/20...,Dec 19
32872,12/19/2017 01:29 PMPosted by MissGunfire12/19/...,(Edited)
32873,12/19/2017 01:24 PMPosted by MissGunfire12/19/...,Dec 19


In [65]:
df_s1500.head(20)

,text,date
0,"A couple months ago, the Damage Done stat used...",Dec 19
1,I believe they said this was a bug.,Dec 19
2,Maybe you should be less concerned about stats...,Dec 19
3,How does this stand now?,Dec 19
4,12/19/2017 04:27 AMPosted by ElfcripplerI beli...,Dec 19
5,There are already a plethora of ways to get ar...,Dec 19
6,12/19/2017 04:23 AMPosted by Shumb0A couple mo...,Dec 19
7,12/19/2017 04:27 AMPosted by ElfcripplerI beli...,Dec 19
8,"I've seen so many threads talking about ""Girl ...",Dec 19
9,It will be true when people stop bullying girl...,Dec 19


In [63]:
df_s1500.tail(20)

,text,date
19891,It's quite good. https://www.youtube.com/watch...,"Nov 13, 2017"
19892,Yup...Makes me sad though. This year is the Ye...,"Nov 13, 2017"
19893,All of his videos are quite good,"Nov 13, 2017"
19894,11/13/2017 12:39 PMPosted by PupsikAll of his ...,"Nov 13, 2017"
19895,“Overwatch is to lootboxes as Resident Evil is...,"Nov 13, 2017"
19896,I'm surprised I haven't seen people jumping to...,"Nov 13, 2017"
19897,I know right?!Jim is right though...,(Edited)
19898,11/13/2017 01:04 PMPosted by ChaosTriggerI'm s...,"Nov 13, 2017"
19899,"I got about 2 seconds in, who is this guy to me?","Nov 13, 2017"
19900,11/13/2017 01:08 PMPosted by idSurgeI got abou...,"Nov 13, 2017"


In [62]:
df_s1500to5k.head(20)

,text,date
0,"Manual ban system, with millions of players? N...","Nov 13, 2017"
1,"CS:GO Overwatch, that's another solution","Nov 13, 2017"
2,"I'm not sure what you mean, like 6 v 6 Soldie...","Nov 13, 2017"
3,11/12/2017 05:40 PMPosted by idSurgeI'm not s...,"Nov 13, 2017"
4,"Oh. Seems like a fine idea, I would imagine it...",(Edited)
5,Maybe they should buff the bad heroes instead ...,"Nov 13, 2017"
6,11/12/2017 05:40 PMPosted by idSurgeI'm not s...,"Nov 13, 2017"
7,people are delusional if they think blizzard c...,(Edited)
8,11/12/2017 05:45 PMPosted by idSurgeOh. Seems ...,"Nov 13, 2017"
9,11/12/2017 05:48 PMPosted by HanzoMaybe they s...,"Nov 13, 2017"


In [58]:
for df in dfs_listed:
    print(df['date'].value_counts())

 (Edited)       821
Feb 20          719
Feb 15          705
Feb 12          583
Feb 16          528
Feb 18          485
Feb 13          479
Feb 11          466
Feb 14          466
Feb 19          384
Feb 17          330
Feb 8           109
Feb 10           79
Feb 9            78
Jan 17           33
Dec 16           29
Nov 4, 2017      27
Jun 24, 2016     27
Jan 10           24
Nov 21, 2017     20
Feb 1            20
May 24, 2017     20
Feb 3            18
Dec 22           18
Oct 16, 2016     18
Dec 10           18
Jan 16           18
Mar 25, 2017     18
Sep 8, 2016      18
Feb 4            17
               ... 
Nov 28, 2017      1
May 26, 2017      1
Feb 2             1
May 17, 2017      1
Jul 11, 2017      1
Aug 30, 2017      1
May 26, 2016      1
Oct 14, 2016      1
Apr 27, 2017      1
Dec 1, 2017       1
Mar 3, 2017       1
Aug 24, 2016      1
Jul 19, 2017      1
Aug 30, 2016      1
Dec 2, 2016       1
Oct 10, 2016      1
Apr 19, 2017      1
Oct 22, 2017      1
Oct 13, 2016      1


In [48]:
df_s5kto10k.head()

,text,date
0,I've been hearing some rumors that Mercy might...,"May 12, 2017"
1,Mercy does not need changes at this moment in ...,"May 12, 2017"
2,On the fence on this one...,"May 12, 2017"
3,It would be interesting to play around with bu...,"May 12, 2017"
4,That leak was full of !@#$. Why would such an ...,"May 12, 2017"


In [17]:
dfp2['date'][4].replace('\t','').replace('\n','')

'Feb 20'

In [294]:
path_to_chromedriver = '../Garage/chromedriver'                          # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path_to_chromedriver)       # Open browser
list_of_dicts = []
saver = './data/full_scrapes/Overwatch_'                                 # Path to save

for forum_page in range(0,1001):                                         # For each page in chunk of posts
    current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page)
    browser.get(current_list)                                        # Go to forum page
    title_list = browser.find_elements_by_class_name("ForumTopic-title")# Define list of topic pages to click
    if forum_page % 100 == 0:                                        # Failsafe saver per 100 pages
        save_posts(list_of_dicts, saver, forum_page)

    for i in range(len(title_list)):                                 # For every title in the list
        title_list[i].click()                                    # Click on Title i
        more_pages = True                                        # True means there's more pages
        print('start dict')
        while more_pages == True:                                # While true that there's more pages...
            title      = browser.find_element_by_class_name('Topic-title').text
            dates      = [e.get_attribute('data-tooltip-content') for e in browser.find_elements_by_class_name('TopicPost-timestamp') if e.text != ' (Edited)']
            words      = [e.text for e in browser.find_elements_by_class_name('TopicPost-bodyContent')]
            nums_dict  = [e.get_attribute('data-topic-post') for e in browser.find_elements_by_class_name('TopicPost')]
            post_num   = [e.get_attribute('id') for e in browser.find_elements_by_class_name('TopicPost')]
            auth_posts = [e.text for e in browser.find_elements_by_class_name('Author-posts') if e.get_attribute('data-toggle') == 'tooltip']
            prof_link  = [e.get_attribute('href') for e in browser.find_elements_by_class_name('Author-avatar ')]
            all_imgs   = [e.get_attribute('src') for e in browser.find_elements_by_css_selector('img')]
            auth_img   = [e for e in all_imgs if 'blznav' not in e and len(e) != 0][1:-1]
            print('dict components defined')
            for post in range(len(words)):                       # For each post in topic
                try:
                    post_dict = {                                    # Creation & Statement of dicts
                        'text'      : words[post],                   # Text of the post
                        'date'      : dates[post],                   # Date of the (unedited) post
                        'nums_dict' : nums_dict[post],               # Author info & votes of the post
                        'post_num'  : post_num[post],                # Post umber in the topic
                        'prof_link' : prof_link[post],               # Link to author profile
                        'auth_img'  : auth_img[post],                # Profile image of author
                        'title'     : title,                         # Title of Topic
                        'forum_page': forum_page                     # Page in the forum
                    }
                except:
                    pass
                try:
                    post_dict['auth_posts'] = auth_posts[post]   # Number of posts author has made
                except:
                    pass
                list_of_dicts.append(post_dict)
                print('dict created and appended')
            more_pages = click_next()                            # Clicks "NEXT" otherwise return False
            print('next clicked')
        browser.get(current_list)                                # Returns to current forum page
        sleep(0.5)                                               # Tiny safety sleep
        print('slept')
        title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine list for freshness
save_posts(list_of_dicts, saver, forum_page)                             # Finally save the posts
df = pd.DataFrame(list_of_dicts, columns = ['text','date'])              # Return the dataframe for looking! :D

start dict
dict components defined
dict created and appended
dict created and appended
next clicked
slept
start dict
dict components defined
dict created and appended
next clicked
slept
start dict
dict components defined
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
next clicked
dict components defined
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created and appended
dict created a

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=51282): Max retries exceeded with url: /session/e7d52587f19ee4dcdc2fe5c5868631d3/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a2ccdee80>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [20]:
dfp2['date'].map(lambda x: x.replace('\t','').replace('\n',''))

0           Feb 20
1           Feb 20
2           Feb 20
3         (Edited)
4           Feb 20
5           Feb 18
6         (Edited)
7           Feb 19
8           Feb 19
9           Feb 19
10          Feb 19
11          Feb 19
12          Feb 19
13          Feb 19
14          Feb 19
15        (Edited)
16          Feb 19
17          Feb 19
18          Feb 19
19          Feb 19
20          Feb 19
21          Feb 19
22        (Edited)
23          Feb 19
24          Feb 19
25          Feb 19
26          Feb 19
27          Feb 19
28          Feb 19
29          Feb 20
           ...    
13682       Feb 14
13683       Feb 14
13684       Feb 14
13685       Feb 14
13686     (Edited)
13687       Feb 14
13688       Feb 14
13689       Feb 14
13690       Feb 14
13691       Feb 14
13692       Feb 14
13693     (Edited)
13694       Feb 14
13695       Feb 14
13696       Feb 14
13697       Feb 14
13698       Feb 14
13699       Feb 14
13700       Feb 14
13701       Feb 14
13702       Feb 14
13703       

In [103]:
soup.find_all('span',{"class" :"ForumTopic-title"})

[<span class="ForumTopic-title" data-toggle="tooltip" data-tooltip-content="Welcome to the General Discussion forum! We encourage you to use this forum to provide feedback and/or discuss your experiences while playing the Overwatch. Community forums work best when participan…">
 
 
 					Welcome to General Discussion - Please Read!
 <i class="statusIcon statusIcon-mobile" data-toggle="tooltip" data-tooltip-content="Locked"></i> </span>,
 <span class="ForumTopic-title" data-toggle="tooltip" data-tooltip-content="To help us get the most of out of your feedback, we ask that you please do not post game-related bugs or technical issues in this forum. To report in-game bugs experienced while playing Overwatch, pl…">
 
 
 					Bug Report and Technical Support Forums
 <i class="statusIcon statusIcon-mobile" data-toggle="tooltip" data-tooltip-content="Locked"></i> </span>,
 <span class="ForumTopic-title" data-toggle="tooltip" data-tooltip-content="Hi everyone, We've continued to see a heavy inf

In [135]:
title_list = browser.find_elements_by_class_name("ForumTopic-title")
for i in range(len(title_list)):
    
for i in range(10):
    sleep(2)
    print(browser.find_element_by_class_name("ForumTopic-title"))
    browser.find_element_by_class_name("ForumTopic-title").click()
    browser.back()

<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.5193284348174689-1")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.34009736080179187-1")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.9742492348653073-1")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.6140948632068146-1")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.48394356344838885-1")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.7130761862957133-1")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.20290966477631822-1")>
<selenium.webdriver.remote.webelement.WebElement (session="cae02938f147fa6edb631d90ee440b6d", element="0.882

In [96]:
path_to_chromedriver = '../Garage/chromedriver'
browser = webdriver.Chrome(executable_path = path_to_chromedriver)

In [97]:
browser.get("https://us.battle.net/forums/en/overwatch/topic/20744334244")

In [98]:
soup = BeautifulSoup(browser.page_source, "lxml")

In [100]:
soup

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport"/>
<title>Are you Kidding Me? - Overwatch Forums</title>
<meta content="website" property="og:type"/>
<meta content="https://us.battle.net/forums/en/overwatch/topic/20744334244" property="og:url"/>
<meta content="Are you Kidding Me? - Overwatch Forums" property="og:title"/>
<meta content="/forums/static/images/social-thumbs/overwatch.png" property="og:image"/>
<meta content="Are you kidding me, Blizzard? After all the changes you made between the second and third BETA sequences, I actually thought you guys cared about balancing your game. Bastion is still WAY TOO STRONK. His turret form has way too much tankiness and range for how much absolutely insane damage he does. Yes, good job taking away his shield. He did NOT, NOT, NOT need to be able to swivel 360 unrestricted (doesn't eve

In [ ]:
def unknown_funct(title_list, list_of_dicts, browser):
    for i in range(len(title_list)):
        try:
            title_list[i].click()                                           # Click on Title i
            soup = BeautifulSoup(browser.page_source, 'lxml')               # Soup of all
            words = soup.find_all('div',{"class" :'TopicPost-bodyContent'}) # Words
            dates = soup.find_all('a',{"class" :'TopicPost-timestamp'})     # Dates
            sleep(0.5)
            
            for post in range(len(words)):
                post_dict = {                                         # Creation & Statement of dicts
                    'text' : words[post].text,
                    'date' : dates[post].text
                }
                list_of_dicts.append(post_dict)

            browser.back()                                            # Back
            title_list = browser.find_elements_by_class_name("ForumTopic-title")
        except:
            print("Woops", forum_page, i)
            save_posts(list_of_dicts, saver, forum_page)

In [152]:
browser.back()

In [154]:
dates = browser.find_elements_by_class_name('TopicPost-timestamp')
for element in browser.find_elements_by_class_name('TopicPost-timestamp'):
    print('------')
    print(element.text)
    print(element.get_attribute('data-tooltip-content'))

------
May 24, 2016
05/24/2016 12:44 AM
------
May 24, 2016
05/24/2016 12:45 AM
------
 (Edited)
MartinRiggs 05/24/2016 12:46 AM
------
May 24, 2016
05/24/2016 12:46 AM
------
May 24, 2016
05/24/2016 12:46 AM
------
May 24, 2016
05/24/2016 12:47 AM
------
May 24, 2016
05/24/2016 12:48 AM
------
May 24, 2016
05/24/2016 12:50 AM
------
May 24, 2016
05/24/2016 12:52 AM
------
May 24, 2016
05/24/2016 12:54 AM
------
May 24, 2016
05/24/2016 12:55 AM
------
May 24, 2016
05/24/2016 12:55 AM
------
May 24, 2016
05/24/2016 12:57 AM
------
May 24, 2016
05/24/2016 12:57 AM
------
May 24, 2016
05/24/2016 01:01 AM
------
May 24, 2016
05/24/2016 01:01 AM
------
 (Edited)
OnTour 05/24/2016 01:05 AM
------
May 24, 2016
05/24/2016 01:11 AM
------
May 24, 2016
05/24/2016 01:11 AM
------
May 24, 2016
05/24/2016 01:12 AM
------
May 24, 2016
05/24/2016 01:13 AM
------
May 24, 2016
05/24/2016 01:15 AM


In [186]:
dates = browser.find_elements_by_class_name('TopicPost-timestamp')
print(len(dates))
edits = []
edit_nums = []
#for element_num in range(len(dates)):
#    if "(Edited)" in dates[element_num].text:
#        print('element', dates[element_num].text)
#        edit_nums.append(element_num)

[element for element in dates if element.text not '(Edited)']
for element in dates:
    if "(Edited)" in element.:
        print('element', element.rect. text)
        #edit_nums.append(element_num)
        dates -= element
        #y = [s for s in x if len(s) == 2]
print(len(edits))
print(edits)
count = 0
#for num in edit_nums:
    
    #edits.append(dates[num].get_attribute('data-tooltip-content'))
#    dates.pop(num + count)
#    count += 1
print(edits)
print(len(dates))


for element in dates:
    print('------')
    print(element.text)
    print(element.get_attribute('data-tooltip-content'))

SyntaxError: invalid syntax (<ipython-input-186-11280649f39f>, line 10)

In [257]:
dates      = [e.get_attribute('data-tooltip-content') for e in browser.find_elements_by_class_name('TopicPost-timestamp') if e.text != ' (Edited)']
words      = [e.text for e in browser.find_elements_by_class_name('TopicPost-bodyContent')]
nums_dict  = [e.get_attribute('data-topic-post') for e in browser.find_elements_by_class_name('TopicPost')]
post_num   = [e.get_attribute('id') for e in browser.find_elements_by_class_name('TopicPost')]
auth_posts = [e.text for e in browser.find_elements_by_class_name('Author-posts') if e.get_attribute('data-toggle') == 'tooltip']
prof_link  = [e.get_attribute('href') for e in browser.find_elements_by_class_name('Author-avatar ')]
all_imgs   = [e.get_attribute('src') for e in browser.find_elements_by_css_selector('img')]
auth_img   = [e for e in all_imgs if 'blznav' not in e and len(e) != 0][1:-1]

In [136]:
elements = browser.find_elements_by_class_name('TopicPost ')
for element in elements:
    print('------')
    #print(element)
    #print(element.get_property('title'))
    #print(element.get_attribute('property'))
    print(element.get_attribute('data-topic-post'))
    print(element.get_attribute('id'))

------
{"id":"207425174325","valueVoted":0,"rank":{"voteUp":0,"voteDown":36},"author":{"id":"196469910876","name":"Dakrath"}}
post-1
------
{"id":"207425174328","valueVoted":0,"rank":{"voteUp":18,"voteDown":1},"author":{"id":"186138967094","name":"MartinRiggs"}}
post-2
------
{"id":"207425274314","valueVoted":0,"rank":{"voteUp":1,"voteDown":1},"author":{"id":"187538495100","name":"Leinhart"}}
post-3
------
{"id":"207425474302","valueVoted":0,"rank":{"voteUp":6,"voteDown":3},"author":{"id":"192868519547","name":"iHazelwood"}}
post-4
------
{"id":"207424374360","valueVoted":0,"rank":{"voteUp":15,"voteDown":0},"author":{"id":"207416589379","name":"sandwhip"}}
post-5
------
{"id":"207425374370","valueVoted":0,"rank":{"voteUp":11,"voteDown":0},"author":{"id":"186013081591","name":"RubberDucky"}}
post-6
------
{"id":"207425374378","valueVoted":0,"rank":{"voteUp":1,"voteDown":18},"author":{"id":"196469910876","name":"Dakrath"}}
post-7
------
{"id":"207425074318","valueVoted":0,"rank":{"voteUp

In [153]:
for element in browser.find_elements_by_class_name('TopicPost-bodyContent'):
    print('------')
    print(element.text)
    #print(element.get_attribute('data-tooltip-content'))

------
Are you kidding me, Blizzard? After all the changes you made between the second and third BETA sequences, I actually thought you guys cared about balancing your game. Bastion is still WAY TOO STRONK. His turret form has way too much tankiness and range for how much absolutely insane damage he does. Yes, good job taking away his shield. He did NOT, NOT, NOT need to be able to swivel 360 unrestricted (doesn't even have to backtrack once he hits a certain point) and the speed at which he can turn means that he might as well be able to shoot in every direction. If the balance on this game isn't fixed within the first 2 weeks of release, I'm asking for a refund. Not paying for another broken game. At least not quietly.
------
Maybe stopping charging into him?
------
Glhf
------
Git gud
------
05/23/2016 05:44 PM
Posted by Dakrath
Are you kidding me, Blizzard? After all the changes you made between the second and third BETA sequences, I actually thought you guys cared about balancing 

In [ ]:
soup.find_all('a',{"class" :'TopicPost-timestamp'})

In [264]:
browser.find_element_by_class_name('Topic-title').text

'Are you Kidding Me?'

In [140]:

for element in browser.find_elements_by_class_name('TopicPost-timestamp'):
    print('------')
    print(element.text)
    print(element.get_attribute('data-tooltip-content'))

------
May 24, 2016
05/24/2016 12:44 AM
------
May 24, 2016
05/24/2016 12:45 AM
------
 (Edited)
MartinRiggs 05/24/2016 12:46 AM
------
May 24, 2016
05/24/2016 12:46 AM
------
May 24, 2016
05/24/2016 12:46 AM
------
May 24, 2016
05/24/2016 12:47 AM
------
May 24, 2016
05/24/2016 12:48 AM
------
May 24, 2016
05/24/2016 12:50 AM
------
May 24, 2016
05/24/2016 12:52 AM
------
May 24, 2016
05/24/2016 12:54 AM
------
May 24, 2016
05/24/2016 12:55 AM
------
May 24, 2016
05/24/2016 12:55 AM
------
May 24, 2016
05/24/2016 12:57 AM
------
May 24, 2016
05/24/2016 12:57 AM
------
May 24, 2016
05/24/2016 01:01 AM
------
May 24, 2016
05/24/2016 01:01 AM
------
 (Edited)
OnTour 05/24/2016 01:05 AM
------
May 24, 2016
05/24/2016 01:11 AM
------
May 24, 2016
05/24/2016 01:11 AM
------
May 24, 2016
05/24/2016 01:12 AM
------
May 24, 2016
05/24/2016 01:13 AM
------
May 24, 2016
05/24/2016 01:15 AM


In [110]:
element_test = browser.find_element_by_css_selector('meta')

In [129]:
soup.find_all('div',{"class" :'Timestamp-details'})[0] # Words

<div class="Timestamp-details">
<a class="TopicPost-timestamp" data-original-title="" data-toggle="tooltip" data-tooltip-content="05/24/2016 12:44 AM" href="#post-1" title="">
						May 24, 2016
					</a>
<span class="TopicPost-rank TopicPost-rank--down" data-original-title="" data-toggle="tooltip" data-tooltip-content="36 Dislikes, 0 Likes" data-topic-post-rank="true" title="">-36</span> </div>

In [126]:
elements = browser.find_elements_by_css_selector('div')
for element in elements:
    print('------')
    print(element.value_of_css_property)
    #print(element.get_property('title'))
    #print(element.get_attribute('property'))
    #print(element.get_attribute('content'))

------
<bound method WebElement.value_of_css_property of <selenium.webdriver.remote.webelement.WebElement (session="cbcae5dead99ca9a41f5bf07dcc1473a", element="0.5621916735748527-8")>>
------
<bound method WebElement.value_of_css_property of <selenium.webdriver.remote.webelement.WebElement (session="cbcae5dead99ca9a41f5bf07dcc1473a", element="0.5621916735748527-9")>>
------
<bound method WebElement.value_of_css_property of <selenium.webdriver.remote.webelement.WebElement (session="cbcae5dead99ca9a41f5bf07dcc1473a", element="0.5621916735748527-10")>>
------
<bound method WebElement.value_of_css_property of <selenium.webdriver.remote.webelement.WebElement (session="cbcae5dead99ca9a41f5bf07dcc1473a", element="0.5621916735748527-11")>>
------
<bound method WebElement.value_of_css_property of <selenium.webdriver.remote.webelement.WebElement (session="cbcae5dead99ca9a41f5bf07dcc1473a", element="0.5621916735748527-12")>>
------
<bound method WebElement.value_of_css_property of <selenium.webdr

In [119]:
elements = browser.find_elements_by_css_selector('meta')
for element in elements:
    print('------')
    #print(element.get_property('title'))
    print(element.get_attribute('property'))
    print(element.get_attribute('content'))

------

None
IE=edge
------

None
width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no
------

og:type
website
------

og:url
https://us.battle.net/forums/en/overwatch/topic/20744334244
------

og:title
Are you Kidding Me? - Overwatch Forums
------

og:image
/forums/static/images/social-thumbs/overwatch.png
------

og:description
Are you kidding me, Blizzard? After all the changes you made between the second and third BETA sequences, I actually thought you guys cared about balancing your game. Bastion is still WAY TOO STRONK. His turret form has way too much tankiness and range for how much absolutely insane damage he does. Yes, good job taking away his shield. He did NOT, NOT, NOT need to be able to swivel 360 unrestricted (doesn't even have to backtrack once he hits a certain point) and the speed at which he can turn means that he might as well be able to shoot in every direction. If the balance on this game isn't fixed within the first 2 weeks of release, I'm askin

In [ ]:
next_button = browser.find_element_by_class_name("stats-table-pagination__next") # Defines the button

next_button.click()                           # Clicks the button

## NBA Example

In [95]:

browser.get("https://stats.nba.com/leaders/")

html = browser.page_source                    # Better than requests.get and will return the html
soup = BeautifulSoup(html, 'lxml')            # Making prev line legible

soup.find_all("td", {"class": "player"})      # Returns all name = td, with class = player

next_button = browser.find_element_by_class_name("stats-table-pagination__next") # Defines the button

next_button.click()                           # Clicks the button

results = requests.get("http://stats.nba.com/leaders/") # There's Java Script running in this browser

results.text

'<!DOCTYPE html>\n<html xmlns:ng="http://angularjs.org" lang="en" ng-app="stats" ng-strict-di stats-version="4.15.5" stats-build-date="2018-11-01 12:43 PM">\n\n  <head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1"><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQECWF5UChAHUlNTBwgBVw=="};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o]);return r}({1:[function(t,n,e){function r(t){try{s.console&&console.log(t)}catch(n){}}var o,i=t("ee"),a=t(16),s={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.console=!0,o.indexOf("dev")!==-1&&(s.dev=!0),o.indexOf("nr_dev")!==-1&&(s.nrDev=!0))}catch(c){}s.nrDev&&i.on("internal-error",f

In [97]:
xpath_button = browser.find_element_by_xpath('/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[1]/div/div/a[2]/i')
xpath_button.click()

In [ ]:
scrapy

In [15]:
url = 'https://us.battle.net/forums/en/overwatch/topic/20762056694'
res = requests.get(url)
res.status_code

200

In [14]:
res.content

b'\n\n\n<html xmlns="http://www.w3.org/1999/xhtml">\n\n<head>\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge"/>\n\t<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no"/>\n\n\t\n\n\t<title>Why would you pick Reinhardt over Orisa? - Overwatch Forums</title>\n\n\t<meta property="og:type" content="website" />\n\t<meta property="og:url" content="https://us.battle.net/forums/en/overwatch/topic/20762056694" />\n\t<meta property="og:title" content="Why would you pick Reinhardt over Orisa? - Overwatch Forums" />\n\t\t<meta property="og:image" content="/forums/static/images/social-thumbs/overwatch.png" />\n\n\t\t<meta property="og:description" content="It&#39;s clear at the moment that Orisa is just superior in almost every way: - Her shield is up for much longer compared to Reinhardt. - She can shoot while the shield is up while Reinhardt can&#39;t do any damage. Orisa spam down a choke can be much more effective at keeping attackers at 

In [19]:
soup = BeautifulSoup(res.content, 'lxml')

In [37]:
soup.find('section').attrs

{'class': ['Topic'],
 'data-topic': '{ "id":20762056694, "lastPosition":30,"forum":{"id":22813879},"isSticky":false,"isFeatured":false,"isLocked":false,"isHidden":false,"isFrozen":false, "isSpam":false, "pollId":0 }',
 'data-user': '{}'}

In [31]:
print(h1.text)
print(h1.attrs)

The Overwatch community forums have moved!
{}


In [75]:
soup.find_all('div',{'class':'Author'})

[<div class="Author" data-topic-post-body-content="true" id=""><a class="Author-avatar " href="https://playoverwatch.com/en-us/career/pc/us/Pigeon-21606"><img alt="" src="https://blzgdapipro-a.akamaihd.net/game/unlocks/0x02500000000012D4.png"/></a><div class="Author-details"> <span class="Author-name">
 <a class="Author-name--profileLink" href="https://playoverwatch.com/en-us/career/pc/us/Pigeon-21606">Pigeon</a>
 </span>
 <span class="Author-posts">
 <a class="Author-posts" data-toggle="tooltip" data-tooltip-content="View Post History" href="/forums/en/overwatch/search?a=Pigeon%2321606">
 908 posts
 </a>
 </span></div></div>,
 <div class="Author" data-topic-post-body-content="true" id=""><a class="Author-avatar " href="https://playoverwatch.com/en-us/career/pc/us/timotmcc-1265"><img alt="" src="https://blzgdapipro-a.akamaihd.net/game/unlocks/0x02500000000013A5.png"/></a><div class="Author-details"> <span class="Author-name">
 <a class="Author-name--profileLink" href="https://playoverw

In [67]:
count = 0
for i in range(len(divs)):
    try:
        print(divs[i].attrs['id'])
        print(divs[i].attrs)
        divs[i].text
    except:
        count += 1

post-1
{'class': ['TopicPost', ''], 'id': 'post-1', 'data-topic-post': '{"id":"207604184492","valueVoted":0,"rank":{"voteUp":8,"voteDown":8},"author":{"id":"207417590476","name":"Pigeon"}}', 'data-topic': '{ "sticky":"false","featured":"false","locked":"false","frozen":"false","hidden":"false","pollId":"0"}'}

{'class': ['Author'], 'id': '', 'data-topic-post-body-content': 'true'}
post-2
{'class': ['TopicPost', ''], 'id': 'post-2', 'data-topic-post': '{"id":"207603084678","valueVoted":0,"rank":{"voteUp":29,"voteDown":2},"author":{"id":"182986383640","name":"timotmcc"}}'}

{'class': ['Author'], 'id': '', 'data-topic-post-body-content': 'true'}
post-3
{'class': ['TopicPost', ''], 'id': 'post-3', 'data-topic-post': '{"id":"207604384709","valueVoted":0,"rank":{"voteUp":8,"voteDown":0},"author":{"id":"207602084749","name":"Stark"}}'}

{'class': ['Author'], 'id': '', 'data-topic-post-body-content': 'true'}
post-4
{'class': ['TopicPost', ''], 'id': 'post-4', 'data-topic-post': '{"id":"2076045

In [71]:
count = 0
for tag in soup.find_all('div'):
    print(tag.text)
















































































































































MenuGamesShopNewsEsportsBlizzcon%SIMPLE_LABEL%Support0My Account%USER_BATTLETAG_FULL%


.Navbar .Navbar-dropdown .Navbar-gamePublishers.Navbar-columns9 .Navbar-gamePublisher.Navbar-columns2 .Navbar-poster {
    width: calc(49.99% - 10px);
}BlizzardOverwatch™World of Warcraft®Hearthstone®Heroes of the Storm™Warcraft® III: Reforged™Diablo® Immortal™Diablo® IIIStarCraft® IIStarCraft®: RemasteredActivisionCall of Duty®: Black Ops 4Destiny® 2More GamesBattle.net AppDownloadsGame Forums.Navbar .Navbar-dropdown .Navbar-esportsLogos .Navbar-esport .Navbar-esportImage {
    width: 14vw;
    height: 14vw
}Overwatch LeagueOverwatch World CupHearthstone Championship TourHeroes Global ChampionshipStarCraft II WCSWorld of Warcraft Arena World ChampionshipMore CompetitionsResourcesBlizzard Arena
Log InAccount SettingsCreate a Free Account%USER_BATTLETAG%%USER_B

In [ ]:
# Scrape this for a potential starting list of curse words for a list of cuss words
https://www.noswearing.com/dictionary/a
Tweet Sentiment Dataset
https://www.kaggle.com/kazanova/sentiment140/home

In [ ]:
links = driver.find_elements_by_xpath("xpath")
links = [link.get_attribute('href') for link in driver.find_elements_by_xpath("xpath")]
for link in links:
    driver.get(link)
for x in range(len(links)):
    links[x].click()
    try:
        driver.implicitly_wait(3)
        DO something
        driver.back()
        print("Mission completed!!")
    except (ElementNotVisibleException, NoSuchElementException):
        driver.back()
        print("No action")


## Data Collection

## Initial Viewing and Cleaning

## Exploritory Data Analysis

## Model Prep

### Model Imports & Initiations

### Data Prep

In [ ]:
#features = 
X = df[features]
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    stratify = y)

## Initial Model and Results

## Model Review & Improvement

## Recycling Bin for Code

In [309]:
# ORIGINAL SCRAPING SAMPLING CODE
#ops = webdriver.chrome.options.Options()
#ops.add_argument('--dns-prefetch-disable')
path = '../Garage/chromedriver'                                         # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path)#, options=ops)         # Open browser
list_of_dicts = []
saver = './data/full_scrapes/Overwatch_Test_' 

for forum_page in range(1,1000):
    try:
        current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page)
        browser.get(current_list)
        title_list = browser.find_elements_by_class_name("ForumTopic-title")
        if forum_page % 100 == 0:
            save_posts(list_of_dicts, saver, forum_page)

        for i in range(len(title_list)):
            try:
                title_list[i].click()                                   # Click on Title i
                more_pages = True                                       # True means there's more pages
                #print('start dict')
                while more_pages == True:                                     # Click on Title i
                    soup = BeautifulSoup(browser.page_source, 'lxml')         # Soup of all
                    words = soup.find_all('div',{"class" :'TopicPost-bodyContent'}) # Words
                    dates = soup.find_all('a',{"class" :'TopicPost-timestamp'})     # Dates
                    for post in range(len(words)):
                        post_dict = {                                         # Creation & Statement of dicts
                            'text' : words[post].text,
                            'date' : dates[post].text
                        }
                        list_of_dicts.append(post_dict)
         
                    more_pages = click_next()                           # Clicks "NEXT" otherwise return False
                    browser.get(current_list)                                            # Back
                    title_list = browser.find_elements_by_class_name("ForumTopic-title")
            except:
                print("Woops", forum_page, i)
                save_posts(list_of_dicts, saver, forum_page)
                sleep(10)
                browser.refresh()
                browser.get(current_list)
    except:
        print("Woops", forum_page)
        save_posts(list_of_dicts, saver, forum_page)
        sleep(30)
        browser.refresh()
        browser.get(current_list)
save_posts(list_of_dicts, saver, forum_page)
df = pd.DataFrame(list_of_dicts, columns = ['text','date'])        

Woops 2 5
Woops 2


KeyboardInterrupt: 

In [314]:
# OLD CODE WITH PROBABLY TOO MANY BROWSER CALLS
ops = webdriver.chrome.options.Options()
ops.add_argument('--dns-prefetch-disable')
path = '../Garage/chromedriver'                                         # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path, options=ops)         # Open browser
list_of_dicts = []
saver = './data/full_scrapes/Overwatch_Pearl_'                          # Path to save

for forum_page in range(0,1001):     # BE CAREFUL WITH THIS             # For each page in chunk of posts
    try:                                                                # Failsafe within forum page
        current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page)
        browser.get(current_list)                                       # Go to forum page
        title_list = browser.find_elements_by_class_name("ForumTopic-title")# Define list of topic pages to click
        if forum_page % 100 == 0 & forum_page != 0:                                       # Failsafe saver per 100 pages
            save_posts(list_of_dicts, saver, forum_page)

        for i in range(len(title_list)):                                # For every title in the list
            try:                                                        # Failsafe in topic page
                title_list[i].click()                                   # Click on Title i
                more_pages = True                                       # True means there's more pages
                #print('start dict')
                #sleep(2)
                while more_pages == True:                               # While true that there's more pages...
                    title      = browser.find_element_by_class_name('Topic-title').text
                    dates      = [e.get_attribute('data-tooltip-content') for e in browser.find_elements_by_class_name('TopicPost-timestamp') if e.text != ' (Edited)']
                    words      = [e.text for e in browser.find_elements_by_class_name('TopicPost-bodyContent')]
                    nums_dict  = [e.get_attribute('data-topic-post') for e in browser.find_elements_by_class_name('TopicPost')]
                    #sleep(2)
                    post_num   = [e.get_attribute('id') for e in browser.find_elements_by_class_name('TopicPost')]
                    auth_posts = [e.text for e in browser.find_elements_by_class_name('Author-posts') if e.get_attribute('data-toggle') == 'tooltip']
                    prof_link  = [e.get_attribute('href') for e in browser.find_elements_by_class_name('Author-avatar ')]
                    all_imgs   = [e.get_attribute('src') for e in browser.find_elements_by_css_selector('img')]
                    auth_img   = [e for e in all_imgs if 'blznav' not in e and len(e) != 0][1:-1]
                    #print('dict components defined')
                    for post in range(len(words)):                      # For each post in topic
                        try:
                            post_dict = {                               # Creation & Statement of dicts
                                'text'      : words[post],              # Text of the post
                                'date'      : dates[post],              # Date of the (unedited) post
                                'nums_dict' : nums_dict[post],          # Author info & votes of the post
                                'post_num'  : post_num[post],           # Post umber in the topic
                                'prof_link' : prof_link[post],          # Link to author profile
                                'auth_img'  : auth_img[post],           # Profile image of author
                                'title'     : title,                    # Title of Topic
                                'forum_page': forum_page}               # Page in the forum
                        except:
                            pass
                        try:
                            post_dict['auth_posts'] = auth_posts[post]  # Number of posts author has made
                        except:
                            pass
                        list_of_dicts.append(post_dict)
                    more_pages = click_next()                           # Clicks "NEXT" otherwise return False
                    #print('next clicked')
                browser.get(current_list)                               # Returns to current forum page
                #sleep(3)                                              # Tiny safety sleep
                #print('slept')
                title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine list for freshness
                
            except:
                print("Woops", forum_page, i)                           # Informs of the problem
                save_posts(list_of_dicts, saver, forum_page)            # Save current posts
                sleep(10)                                               # Safety sleep
                browser.refresh()                                       # Safety Refresh
                browser.get(current_list)                               # Return to forum page
                title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine for freshness
    except:
        print("Woops", forum_page)                                      # Informs that there's a problem
        save_posts(list_of_dicts, saver, forum_page)                    # Save current posts
        sleep(30)                                                       # Long Safety Sleep
        browser.refresh()                                               # Safety Refresh
        browser.get(current_list)                                       # Return to forum page
        
save_posts(list_of_dicts, saver, forum_page)                            # Finally save the posts
df = pd.DataFrame(list_of_dicts, columns = ['text','date'])             # Return the dataframe for looking! :D

Woops 0 20
Woops 0


TimeoutException: Message: timeout
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [315]:
def click_next():
    more_pages = True
    try:
        nexts = []
        for element in browser.find_elements_by_class_name('Button-content'):
            if element.text == 'NEXT':
                nexts.append(element)
        nexts[1].click()
        #not_last_page = True
    except:
        more_pages = False
    return more_pages

In [438]:
#ops = webdriver.chrome.options.Options()
#ops.add_argument('--dns-prefetch-disable')
path = '../Garage/chromedriver'                                         # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path)#, options=ops)         # Open browser
list_of_dicts = []
saver = './data/full_scrapes/Overwatch_Test_' 

for forum_page in range(1,1000):
    try:
        current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page)
        browser.get(current_list)
        title_list = browser.find_elements_by_class_name("ForumTopic-title")
        if forum_page % 100 == 0:
            save_posts(list_of_dicts, saver, forum_page)

        for i in range(len(title_list)):                                # For every title in the list
            try:                                                        # Failsafe in topic page
                title_list[i].click()                                   # Click on Title i
                more_pages = True                                       # True means there's more pages
                next_page = 2
                current_topic = browser.current_url
                while more_pages == True:                               # While true that there's more pages...
                    soup       = BeautifulSoup(browser.page_source, 'lxml')         # Soup of words
                    title      = soup.find(attrs={'class':'Topic-title'}).contents
                    TopicPosts = soup.find_all(attrs = {"class" :'TopicPost'})
                    dates      = [e for e in soup.find_all('a',{"class" :'TopicPost-timestamp'}) if e.contents != ['\n\t\t\t\t\t\t\t\xa0(Edited)\n']]
                    words      = soup.find_all('div',{"class" :'TopicPost-bodyContent'})
                    auth_posts = [e.contents for e in soup.find_all('a',attrs = {'class' :'Author-posts'})]
                    prof_link  = soup.find_all(attrs = {"class" :'Author-avatar '})
                    all_imgs   = [e.get_attribute('src') for e in browser.find_elements_by_css_selector('img')]
                    auth_img   = [e for e in all_imgs if 'blznav' not in e and len(e) != 0][1:-1]
                    for post in range(len(words)):                      # For each post in topic
                        try:
                            post_dict = {                               # Creation & Statement of dicts
                                'text'      : words[post].contents,                     # Text of the post
                                'date'      : dates[post].attrs['data-tooltip-content'],# Date of the (unedited) post
                                'ids_dict'  : TopicPosts[post].attrs['data-topic-post'],# Author info & votes of the post
                                'post_num'  : TopicPosts[post].attrs['id'] ,            # Post number in the topic
                                'statuses'  : TopicPosts[post].attrs['data-topic'],
                                'prof_link' : prof_link[post].attrs['href'],            # Link to author profile
                                'auth_img'  : auth_img[post],           # Profile image of author
                                'title'     : title,                    # Title of Topic
                                'forum_page': forum_page}               # Page in the forum
                        except:
                            pass
                        try:
                            post_dict['auth_posts'] = auth_posts[post]  # Number of posts author has made
                        except:
                            pass
                        list_of_dicts.append(post_dict)
                    try:
                        browser.get(current_topic + '?page=' + str(next_page))
                        soup = BeautifulSoup(browser.page_source, 'lxml')
                        if soup.find(attrs = {'class':'error-type'}).contents[0] == '404':
                            more_pages = False
                        else:
                            next_page += 1
                    except:
                        more_pages = False
                browser.get(current_list)                               # Returns to current forum page
                #sleep(3)                                                # Tiny safety sleep
                title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine list for freshness
                
            except:
                print("Woops", forum_page, i)                           # Informs of the problem
                save_posts(list_of_dicts, saver, forum_page)            # Save current posts
                sleep(10)                                               # Safety sleep
                browser.refresh()                                       # Safety Refresh
                browser.get(current_list)                               # Return to forum page
                title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine for freshness
    except:
        print("Woops", forum_page)
        save_posts(list_of_dicts, saver, forum_page)
        sleep(30)
        browser.refresh()
        browser.get(current_list)
save_posts(list_of_dicts, saver, forum_page)
df = pd.DataFrame(list_of_dicts, columns = ['text','date'])        

Woops 1


KeyboardInterrupt: 

In [450]:
# THIS is our new official one??
path = '../Garage/chromedriver'                                         # Path to Chromedriver
browser = webdriver.Chrome(executable_path = path)#, options=ops)         # Open browser
list_of_dicts = []
saver = './data/full_scrapes/Overwatch_Test_' 

for forum_page in range(72,1000):
    try:
        current_list = 'https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page)
        browser.get(current_list)
        title_list = browser.find_elements_by_class_name("ForumTopic-title")
        if forum_page % 100 == 0:
            save_posts(list_of_dicts, saver, forum_page)

        for i in range(len(title_list)):                                # For every title in the list
            try:                                                        # Failsafe in topic page
                title_list[i].click()                                   # Click on Title i
                more_pages = True                                       # True means there's more pages
                next_page = 2
                current_topic = browser.current_url
                while more_pages == True:                               # While true that there's more pages...
                    soup       = BeautifulSoup(browser.page_source, 'lxml')         # Soup of words
                    title      = soup.find(attrs={'class':'Topic-title'}).contents
                    TopicPosts = soup.find_all(attrs = {"class" :'TopicPost'})
                    dates      = [e for e in soup.find_all('a',{"class" :'TopicPost-timestamp'}) if e.contents != ['\n\t\t\t\t\t\t\t\xa0(Edited)\n']]
                    words      = soup.find_all('div',{"class" :'TopicPost-bodyContent'})
                    auth_posts = [e.contents for e in soup.find_all('a',attrs = {'class' :'Author-posts'})]
                    prof_link  = soup.find_all(attrs = {"class" :'Author-avatar '})
                    all_imgs   = [e.get_attribute('src') for e in browser.find_elements_by_css_selector('img')]
                    auth_img   = [e for e in all_imgs if 'blznav' not in e and len(e) != 0][1:-1]
                    for post in range(len(words)):                      # For each post in topic
                        try:
                            post_dict = {                               # Creation & Statement of dicts
                            'text'      : words[post].contents,                     # Text of the post
                            'date'      : dates[post].attrs['data-tooltip-content'],# Date of the (unedited) post
                            'ids_dict'  : TopicPosts[post].attrs['data-topic-post'],# Author info & votes of the post
                            'post_num'  : TopicPosts[post].attrs['id'] ,            # Post number in the topic
                            'auth_img'  : auth_img[post],           # Profile image of author
                            'title'     : title,                    # Title of Topic
                            'forum_page': forum_page}               # Page in the forum
                        except:
                            pass
                        try:
                            post_dict['statuses']   = TopicPosts[post].attrs['data-topic']
                            post_dict['prof_link']  = prof_link[post].attrs['href']
                            post_dict['auth_posts'] = auth_posts[post]  # Number of posts author has made
                        except:
                            pass
                        list_of_dicts.append(post_dict)
                    try:
                        browser.get(current_topic + '?page=' + str(next_page))
                        soup = BeautifulSoup(browser.page_source, 'lxml')
                        #print('attempted new page')
                        if soup.find(attrs = {'class':'error-type'}).contents[0] == '404':
                            #print("got a 404")
                            more_pages = False
                        #else:
                        #    print("there is another page and we added 1")
                        #    next_page += 1
                    except:
                        next_page += 1
                browser.get(current_list)                               # Returns to current forum page
                #sleep(3)                                                # Tiny safety sleep
                title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine list for freshness
            except KeyboardInterrupt:
                print("Canceled!")
            except:
                print("TimedOut on Forum Section:", forum_page,"Topic number:", i, "Topic Page:", next_page-1)                           # Informs of the problem
                save_posts(list_of_dicts, saver, forum_page)            # Save current posts
                sleep(10)                                               # Safety sleep
                browser.refresh()                                       # Safety Refresh
                browser.get(current_list)                               # Return to forum page
                title_list = browser.find_elements_by_class_name("ForumTopic-title") # Redefine for freshness
            #except:
            #    print('SOMETHING ELSE WENT WRONG')
            #    pass
    except KeyboardInterrupt:
        print("Canceled!")
    except:
        print("TimedOut on Forum Section:", forum_page)
        save_posts(list_of_dicts, saver, forum_page)
        sleep(30)
        browser.refresh()
        browser.get(current_list)
    #except:
    #    print("SOMETHING ELSE WENT WRONG")
    #    pass
save_posts(list_of_dicts, saver, forum_page)
df = pd.DataFrame(list_of_dicts, columns = ['text','date'])        

TimedOut on Forum Section: 72 Topic number: 32 Topic Page: 1
TimedOut on Forum Section: 72


TimeoutException: Message: timeout
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.43.600229 (3fae4d0cda5334b4f533bede5a4787f7b832d052),platform=Mac OS X 10.14.0 x86_64)


In [422]:
[e.contents for e in soup.find_all('a',{"class" :'TopicPost-timestamp'}) if e.contents != ['\n\t\t\t\t\t\t\t\xa0(Edited)\n']]

[['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t'],
 ['\n\t\t\t\t\t\tMay 24, 2016\n\t\t\t\t\t']]

In [4]:
# THIS is just before deleting a bunch of junk
list_of_dicts = []
saver = './data/full_scrapes/Overwatch_Test_' 

for forum_page in range(0,100):
    #try:
    topic_keys = urls_with_numbers('https://us.battle.net/forums/en/overwatch/22813879/?page='+ str(forum_page))
    if forum_page % 100 == 0:
        save_posts(list_of_dicts, saver, forum_page) 
    for topic_stats in topic_keys:                                # For every title in the list
        for page in range(topic_stats[1]):
            page += 1               # TRY "for page +1 in range(topic_stats[1]):" later
            if page > 1:
                URL = topic_stats[0] + '?page=' + str(page)
            else:
                URL = topic_stats[0]
            page_soup  = get_html(URL)         # Soup of words
            title      = page_soup.find(attrs={'class':'Topic-title'}).contents
            dates      = [e for e in page_soup.find_all('a',{"class" :'TopicPost-timestamp'}) if e.contents != ['\n\t\t\t\t\t\t\t\xa0(Edited)\n']]
            words      = page_soup.find_all('div',{"class" :'TopicPost-bodyContent'})
            TopicPosts = page_soup.find_all(attrs = {"class" :'TopicPost'})
            auth_posts = [e.contents for e in page_soup.find_all('a',attrs = {'class' :'Author-posts'})]
            prof_link  = page_soup.find_all(attrs = {"class" :'Author-avatar '})
        for page in range(topic_info[1]):                      # For each post in topic

            turn_to_dict(post)
            list_of_dicts.append(post_dict)
        try:
            resoup = get_html(title_list[i] + '?page=' + str(next_page))
            if resoup.find(attrs = {'class':'error-type'}).contents[0] == '404':
                more_pages = False
        except:
            next_page += 1
        resoup = get_html(current_list)                               # Returns to current forum page
        #except KeyboardInterrupt:
        #    print("Canceled!")
        #except:
        #    print("TimedOut on Forum Section:", forum_page,"Topic number:", i, "Topic Page:", next_page-1)                           # Informs of the problem
        #    save_posts(list_of_dicts, saver, forum_page)            # Save current posts
        #    sleep(10)    
        print("Success!:", forum_page,"Topic number:", i, "Topic Page:", next_page-1)
    #except KeyboardInterrupt:
    #    print("Canceled!")
    #except:
    #    print("TimedOut on Forum Section:", forum_page)
    #    save_posts(list_of_dicts, saver, forum_page)
    #    sleep(30)
    #    resoup = get_html(current_list)
save_posts(list_of_dicts, saver, forum_page)
df = pd.DataFrame(list_of_dicts, columns = ['text','date'])        

AttributeError: 'NoneType' object has no attribute 'contents'

In [413]:
for i in soup.find_all('a',{"class" :'TopicPost-timestamp'}):
    print(i)

<a class="TopicPost-timestamp" data-toggle="tooltip" data-tooltip-content="05/24/2016 12:44 AM" href="#post-1">
						May 24, 2016
					</a>
<a class="TopicPost-timestamp" data-toggle="tooltip" data-tooltip-content="05/24/2016 12:45 AM" href="#post-2">
						May 24, 2016
					</a>
<a class="TopicPost-timestamp" data-toggle="tooltip" data-tooltip-content="MartinRiggs 05/24/2016 12:46 AM" href="#post-2">
							 (Edited)
</a>
<a class="TopicPost-timestamp" data-toggle="tooltip" data-tooltip-content="05/24/2016 12:46 AM" href="#post-3">
						May 24, 2016
					</a>
<a class="TopicPost-timestamp" data-toggle="tooltip" data-tooltip-content="05/24/2016 12:46 AM" href="#post-4">
						May 24, 2016
					</a>
<a class="TopicPost-timestamp" data-toggle="tooltip" data-tooltip-content="05/24/2016 12:47 AM" href="#post-5">
						May 24, 2016
					</a>
<a class="TopicPost-timestamp" data-toggle="tooltip" data-tooltip-content="05/24/2016 12:48 AM" href="#post-6">
						May 24, 2016
					</a>
<a class

http://cryptocats.thetwentysix.io/
    1.) Scrape entire page of cats including
        Cat #
        Cat Status (claimed or not)
        Cat URL
    2.)Identify the navigation items "origin 12", Austrailian Blockchain" and "December"
    3.) Select each of these buttons, automate a click action to each of them
    4.) Run your scraping process from #1, appending the "release type", corresponding to each of these release filters